#  Auto-related costs: based on CES (PUMD weighted estimates of TX region)

### Preparation:import needed package 

In [1]:
# a handy trick to get rid of deprecation warnings
import warnings
warnings.filterwarnings("ignore") 

In [2]:
from configparser import ConfigParser
import dataframe_image as dfi

# it is used for ACS data 
import matplotlib.pyplot as plt
import pandas as pd
from census import Census
from us import states
import pyproj
import geopandas as gpd
import numpy as np

In [3]:
# it is mainly used for producing CES pumd interview data
import os #this is used for importing all needed files in your working directory
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import glob

In [4]:
# It is applied to controlvia pandas' display options (converting exponent or scientific number into float)
pd.set_option('display.float_format', '{:.2f}'.format) 

### Preparation: updated stored files or links

In [5]:
# Read all environment variables
config = ConfigParser()
config.readfp(open(r'Config.py'))

census_api = config.get('General',"census_api") # api key

ces = config.get('Auto',"ces") # CES interview data
tx_wgt = config.get('Auto',"tx_wgt")#state-level weights with the CES public-use microdata
vmt = config.get('Auto',"vmt")#CAMPO all-purpose VMT/hh 

In [6]:
# Set ACS API key
c = Census(census_api) 

In [7]:
## Read CES interview data

#  Store the CES path to the directory containing the five quarterly FMLI files (interview folder)
fmli_files = [os.path.join(ces, f) for f in os.listdir(ces) if f.startswith("fmli")]

# read the files and concate all files via row binds 
li = []

for filename in fmli_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

fmli = pd.concat(li, axis=0, ignore_index=True)
fmli = pd.DataFrame(fmli)

fmli.shape #check the number of rows and columns

(26903, 826)

In [8]:
## lOAD state-level weights with the CES public-use microdata (PUMD)
tx_wgt = pd.read_excel(tx_wgt)

In [9]:
# Load CAMPO all-purpose VMT/hh file (this one is calculated by "3.1 VMT for auto use" )
vmt =pd.read_csv (vmt)

##  A) Working on CES interview data to produce all expenditure variables

In [10]:
fmli.columns = fmli.columns.str.lower() # Change all column names to lower case

#Keep only the necessary variables
fmli = fmli [["newid", "state", "psu", "qintrvyr", "qintrvmo", "fincbtxm", 
                                            "vehq", "age_ref", "mainrpcq","mainrppq", 
                                            "evehpurc", "evehpurp","vehfincq","vehfinpq","vehinscq","vehinspq",
                                            "vrntlocq","vrntlopq","gasmocq","gasmopq","totexpcq","totexppq"]]

fmli.tail()   # check data

,newid,state,psu,qintrvyr,qintrvmo,fincbtxm,vehq,age_ref,mainrpcq,mainrppq,...,vehfincq,vehfinpq,vehinscq,vehinspq,vrntlocq,vrntlopq,gasmocq,gasmopq,totexpcq,totexppq
26898,4352031,17.00,S23A,2020,3,48800.00,1,62,0.00,0.00,...,191.00,0.00,300.00,0.00,26.67,13.33,70.00,35.00,5491.50,3290.75
26899,4352081,6.00,S49A,2020,3,48592.20,1,88,0.00,0.00,...,61.00,31.00,540.00,0.00,0.00,0.00,240.00,120.00,8799.23,3657.12
26900,4352121,25.00,S11A,2020,3,38820.00,1,79,0.00,0.00,...,0.00,0.00,0.00,591.00,0.00,0.00,0.00,0.00,8150.50,4326.25
26901,4352174,28.00,NaN,2020,3,17400.00,2,26,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,240.00,120.00,2638.00,1805.00
26902,4352181,17.00,S23A,2020,3,47389.20,1,31,60.00,0.00,...,0.00,0.00,104.00,0.00,40.00,20.00,60.00,30.00,6066.30,3167.15


In [11]:
      # Ensure that the data type or character length of the variable is correct
fmli ["state"] = fmli ["state"].replace(np.nan, 0) #convert "nan" value in state columns to 0, so as to convert the entire column to string type in the next step
fmli ["state"] = fmli.state.astype(int)
fmli ["state"] = fmli.state.astype(str).str.pad(width = 2, side ='left',fillchar='0')

fmli ["newid"] = fmli.newid.astype(str).str.pad(width = 8, side ='left',fillchar='0')

fmli ["qintrvyr"] = fmli.qintrvyr.astype(int)
fmli ["qintrvmo"] = fmli.qintrvmo.astype(int)

In [12]:
# rename the variables to make it more intuitive

fmli.rename(columns = {'fincbtxm':'income', 'vehq':'num_car','age_ref':'age'}, inplace = True)


In [13]:
# Store variables that keep the calendar year expenditure values for each
  # CU depending on the time of the interview
fmli ["outlay_purchase"] = np.where(fmli ["qintrvyr"] == 2020,  # if we focus on 2019 calender year,  here should be 2020
                                    fmli ["evehpurp"], 
                                    np.where(fmli ["qintrvmo"] <= 3, fmli ["evehpurc"], fmli ["evehpurc"] + fmli ["evehpurp"]))
fmli ["finance"] = np.where(fmli ["qintrvyr"] == 2020, fmli ["vehfinpq"], 
                            np.where(fmli ["qintrvmo"] <= 3, fmli ["vehfincq"], fmli ["vehfincq"] + fmli ["vehfinpq"])) 
fmli ["insurance"] = np.where(fmli ["qintrvyr"] == 2020, fmli ["vehinspq"], 
                                    np.where(fmli ["qintrvmo"] <= 3, fmli ["vehinscq"], fmli ["vehinscq"] + fmli ["vehinspq"]))

fmli ["service"] = np.where(fmli ["qintrvyr"] == 2020, fmli ["vrntlopq"], 
                                    np.where(fmli ["qintrvmo"] <= 3, fmli ["vrntlocq"], fmli ["vrntlocq"] + fmli ["vrntlopq"]))

fmli ["repair"] = np.where(fmli ["qintrvyr"] == 2020, fmli ["mainrppq"], 
                                    np.where(fmli ["qintrvmo"] <= 3, fmli ["mainrpcq"], fmli ["mainrpcq"] + fmli ["mainrppq"]))

fmli ["gas"] = np.where(fmli ["qintrvyr"] == 2020, fmli ["gasmopq"], 
                                    np.where(fmli ["qintrvmo"] <= 3, fmli ["gasmocq"], fmli ["gasmocq"] + fmli ["gasmopq"]))

fmli ["tot_exp"] = np.where(fmli ["qintrvyr"] == 2020, fmli ["totexppq"], 
                                    np.where(fmli ["qintrvmo"] <= 3, fmli ["totexpcq"], fmli ["totexpcq"] + fmli ["totexppq"]))

In [14]:
  # Drop the "cq" "pq","rc","rp" variables
fmli = fmli.loc[:, ~fmli.columns.str.endswith('rc','rp')]

In [15]:
fmli = fmli.loc[:, ~fmli.columns.str.endswith("cq")]
fmli = fmli.loc[:, ~fmli.columns.str.endswith("pq")]
fmli = fmli.loc[:, ~fmli.columns.str.endswith("rc")]
fmli = fmli.loc[:, ~fmli.columns.str.endswith("rp")]

In [16]:
fmli.head()

,newid,state,psu,qintrvyr,qintrvmo,income,num_car,age,outlay_purchase,finance,insurance,service,repair,gas,tot_exp
0,03891134,36,S12A,2019,1,73720.00,1,63,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,03891154,48,S37A,2019,1,12000.00,2,33,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,03891174,25,NaN,2019,1,20000.00,0,50,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,03891224,00,NaN,2019,1,130500.00,5,56,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,03891234,36,S12A,2019,1,27028.40,0,41,0.00,0.00,0.00,0.00,0.00,0.00,0.00


###  A.1) select the sub-sections of the number of vehicles owned by households

In [17]:
# select "num_car" = 0
fmli_car0 = fmli[fmli['num_car'] == 0]

In [18]:
# select "num_car" = 1

fmli_car1 = fmli[fmli['num_car'] == 1]

In [19]:
# select "num_car" = 2
fmli_car2 = fmli[fmli['num_car'] == 2]

In [20]:
# select "num_car" >= 3
fmli_car3 = fmli[fmli['num_car'] >= 3]

## B. Use the State (TX) Weights on the CES Public-Use Microdata

In [21]:
tx_wgt.columns = tx_wgt.columns.str.lower() # Change all column names to lower case

In [22]:
 # Ensure that the data type or character length of the variable is correct
tx_wgt ["state"] = tx_wgt.state.astype(str).str.pad(width = 2, side ='left',fillchar='0')

tx_wgt ["newid"] = tx_wgt.newid.astype(str).str.pad(width = 8, side ='left',fillchar='0')

tx_wgt ["qintrvyr"] = tx_wgt.qintrvyr.astype(int)
tx_wgt ["qintrvmo"] = tx_wgt.qintrvmo.astype(int)

* create variables with the number of months in scope and a population weight, based on the months in scope for a given comsumer unit (CU)

In [23]:
tx_wgt ["mo_scope"] = np.where(tx_wgt ["qintrvyr"] == 2020,  # if we focus on 2019 calender year,  here should be 2020
                                    4 - tx_wgt ["qintrvmo"], 
                                    np.where(tx_wgt ["qintrvmo"] <= 3, tx_wgt ["qintrvmo"] -1, 3))

In [24]:
tx_wgt ["population"] = (tx_wgt["txwgt"] / 4) * (tx_wgt["mo_scope"] / 3)

In [25]:
tx_wgt.tail()

,newid,state,qintrvyr,qintrvmo,psu,txwgt,mo_scope,population
1616,04348231,48,2020,3,NaN,41273.53,1,3439.46
1617,04348891,48,2020,3,NaN,36358.13,1,3029.84
1618,04349011,48,2020,3,S37A,39532.01,1,3294.33
1619,04349701,48,2020,3,S37B,27954.88,1,2329.57
1620,04350061,48,2020,3,NaN,48146.19,1,4012.18


## C. Join sub-section info of the number of vehicles owned by households(fmli_car0 - 3) and weights file (tx_wgt), to produce income segregation of different level of car ownership and correpsonding weighted expenditure

In [26]:
# Generate a table of the calendar year means for each expenditure category and keep the sum of the population weight
tx_wgt.newid = tx_wgt.newid.astype(str)

tx_wtexp0 = fmli_car0.merge(tx_wgt, on=("newid", "state", "psu", "qintrvyr", "qintrvmo"), how='inner')
tx_wtexp1 = fmli_car1.merge(tx_wgt, on=("newid", "state", "psu", "qintrvyr", "qintrvmo"), how='inner')
tx_wtexp2 =fmli_car2.merge(tx_wgt, on=("newid", "state", "psu","qintrvyr", "qintrvmo"), how='inner')
tx_wtexp3 = fmli_car3.merge(tx_wgt, on=("newid", "state", "psu", "qintrvyr", "qintrvmo"), how='inner')

In [27]:
# Multiply each expenditure variable by the TX weight to get a weighted expenditure for each CU
tx_wtexp0[["outlay_purchase","insurance","finance", "repair", "service","gas","tot_exp"]] = tx_wtexp0[["outlay_purchase","insurance","finance","repair", "service","gas","tot_exp"]].multiply(tx_wtexp0["txwgt"], axis="index")
tx_wtexp1[["outlay_purchase","insurance","finance", "repair", "service","gas","tot_exp"]] = tx_wtexp1[["outlay_purchase","insurance","finance","repair", "service","gas","tot_exp"]].multiply(tx_wtexp1["txwgt"], axis="index")
tx_wtexp2[["outlay_purchase","insurance","finance", "repair", "service","gas","tot_exp"]] = tx_wtexp2[["outlay_purchase","insurance","finance","repair", "service","gas","tot_exp"]].multiply(tx_wtexp2["txwgt"], axis="index")
tx_wtexp3[["outlay_purchase","insurance","finance", "repair", "service","gas","tot_exp"]] = tx_wtexp3[["outlay_purchase","insurance","finance","repair", "service","gas","tot_exp"]].multiply(tx_wtexp3["txwgt"], axis="index")


In [28]:
#assign income segregation for each car-ownership group

tx_wtexp0["incomeG"] = np.select(
    [
        tx_wtexp0['income'].between(0, 20000, inclusive='left'), 
        tx_wtexp0['income'].between(20000, 39999, inclusive='both'),
        tx_wtexp0['income'].between(40000, 59999, inclusive='both'), 
        tx_wtexp0['income'].between(60000, 99999, inclusive='both'),
        tx_wtexp0['income'].between(100000, 100000000, inclusive='left')
    ], 
    [
        '1', 
        '2',
        '3', 
        '4',
        '5'
    ]
)

tx_wtexp1["incomeG"] = np.select(
    [
        tx_wtexp1['income'].between(0, 20000, inclusive='left'), 
        tx_wtexp1['income'].between(20000, 39999, inclusive='both'),
        tx_wtexp1['income'].between(40000, 59999, inclusive='both'), 
        tx_wtexp1['income'].between(60000, 99999, inclusive='both'),
        tx_wtexp1['income'].between(100000, 100000000, inclusive='left')
    ], 
    [
        '1', 
        '2',
        '3', 
        '4',
        '5'
    ]
)

tx_wtexp2["incomeG"] = np.select(
    [
        tx_wtexp2['income'].between(0, 20000, inclusive='left'), 
        tx_wtexp2['income'].between(20000, 39999, inclusive='both'),
        tx_wtexp2['income'].between(40000, 59999, inclusive='both'), 
        tx_wtexp2['income'].between(60000, 99999, inclusive='both'),
        tx_wtexp2['income'].between(100000, 100000000, inclusive='left')
    ], 
    [
        '1', 
        '2',
        '3', 
        '4',
        '5'
    ]
)

tx_wtexp3["incomeG"] = np.select(
    [
        tx_wtexp3['income'].between(0, 20000, inclusive='left'), 
        tx_wtexp3['income'].between(20000, 39999, inclusive='both'),
        tx_wtexp3['income'].between(40000, 59999, inclusive='both'), 
        tx_wtexp3['income'].between(60000, 99999, inclusive='both'),
        tx_wtexp3['income'].between(100000, 100000000, inclusive='left')
    ], 
    [
        '1', 
        '2',
        '3', 
        '4',
        '5'
    ]
)


In [29]:
# check whether there has NA or 0 in our new created variable

tx_wtexp0["incomeG"].eq(0).any().any() 
tx_wtexp0["incomeG"].isnull().values.any()

tx_wtexp1["incomeG"].eq(0).any().any() 
tx_wtexp1["incomeG"].isnull().values.any()

tx_wtexp1["incomeG"].eq(0).any().any() 
tx_wtexp1["incomeG"].isnull().values.any()

tx_wtexp2["incomeG"].eq(0).any().any() 
tx_wtexp2["incomeG"].isnull().values.any()

tx_wtexp3["incomeG"].eq(0).any().any() 
tx_wtexp3["incomeG"].isnull().values.any()

False

## D. Obtain demographic information (weighted income) on the number of cars owned by individual at each income level 

#### 0 car

In [30]:
tx_wtexp0 ["incomeW"] = tx_wtexp0 ["income"]* tx_wtexp0 ["txwgt"] #assign weighted income

veh_num0 = tx_wtexp0.groupby('incomeG', as_index=False).agg(num_car=('num_car','mean'), 
                                            incomeW= ('incomeW','sum'),
                                            txwgt= ('txwgt','sum'),                
                                            age=('age','mean'))


veh_num0



,incomeG,num_car,incomeW,txwgt,age
0,1,0.00,19171463080.45,2283942.91,51.64
1,2,0.00,25194163214.30,882613.76,52.60
2,3,0.00,34306679795.59,672561.89,43.90
3,4,0.00,28754682705.64,365032.12,28.38
4,5,0.00,75054977556.05,286904.38,46.90


#### One car

In [31]:
tx_wtexp1 ["incomeW"] = tx_wtexp1 ["income"]* tx_wtexp1 ["txwgt"] #assign weighted income

veh_num1 = tx_wtexp1.groupby('incomeG', as_index=False).agg(num_car=('num_car','mean'), 
                                            incomeW= ('incomeW','sum'),
                                            txwgt= ('txwgt','sum'),                
                                            age=('age','mean'))


veh_num1


,incomeG,num_car,incomeW,txwgt,age
0,1,1.00,45539401567.78,4721990.70,54.38
1,2,1.00,157211766089.81,5382597.10,50.95
2,3,1.00,161084202846.73,3319951.54,44.28
3,4,1.00,232284410470.00,3087732.18,42.56
4,5,1.00,349436430714.38,2372994.65,45.94


#### Two cars

In [32]:
tx_wtexp2 ["incomeW"] = tx_wtexp2 ["income"]* tx_wtexp2 ["txwgt"] #assign weighted income

veh_num2 = tx_wtexp2.groupby('incomeG', as_index=False).agg( num_car=('num_car','mean'), 
                                            incomeW= ('incomeW','sum'),
                                            txwgt= ('txwgt','sum'),                
                                            age=('age','mean'))

veh_num2

,incomeG,num_car,incomeW,txwgt,age
0,1,2.00,13830971886.71,1339698.08,63.04
1,2,2.00,95403327178.95,3311177.47,57.14
2,3,2.00,158125692219.41,3239417.16,50.23
3,4,2.00,327130743492.11,4304849.39,45.92
4,5,2.00,1334671786769.97,6130104.50,46.45


#### Three or more cars

In [33]:
tx_wtexp3 ["incomeW"] = tx_wtexp3 ["income"]* tx_wtexp3 ["txwgt"] #assign weighted income

veh_num3 = tx_wtexp3.groupby('incomeG', as_index=False).agg( num_car=('num_car','mean'), 
                                            incomeW= ('incomeW','sum'),
                                            txwgt= ('txwgt','sum'),                
                                            age=('age','mean'))
veh_num3.drop(index=veh_num3.index[0], 
        axis=0, 
        inplace=True)
veh_num3

,incomeG,num_car,incomeW,txwgt,age
1,1,3.00,3074616901.76,279433.80,51.00
2,2,3.51,43035431756.31,1423262.87,61.35
3,3,3.72,66515148707.41,1350220.36,54.13
4,4,3.94,184851279487.83,2312025.17,54.19
5,5,3.68,1154923660398.16,5860504.72,49.22


In [34]:
#calculate statistics 
veh_num0 ["average_veh"] = veh_num0 ["num_car"]  # the number of cars in each group
veh_num1 ["average_veh"] = veh_num1 ["num_car"]  # the number of cars in each group
veh_num2 ["average_veh"] = veh_num2 ["num_car"]  # the number of cars in each group
veh_num3 ["average_veh"] = veh_num3 ["num_car"]   # the number of cars in each group

veh_num0 ["income"] = veh_num0 ["incomeW"] / veh_num0 ["txwgt"] # weighted income in each group
veh_num1 ["income"] = veh_num1 ["incomeW"] / veh_num1 ["txwgt"] # weighted income in each group
veh_num2 ["income"] = veh_num2 ["incomeW"] / veh_num2 ["txwgt"] # weighted income in each group
veh_num3 ["income"] = veh_num3 ["incomeW"] / veh_num3 ["txwgt"] # weighted income in each group

veh_num0= veh_num0.drop (columns=["txwgt","incomeW"]) #drop unnecessary columns
veh_num1= veh_num1.drop (columns=["txwgt","incomeW"]) #drop unnecessary columns
veh_num2= veh_num2.drop (columns=["txwgt","incomeW"]) #drop unnecessary columns
veh_num3= veh_num3.drop (columns=["txwgt","incomeW"]) #drop unnecessary columns

#formatting
veh_num0 ["age"] = veh_num0 ["age"].astype(int)
veh_num1 ["age"] = veh_num1 ["age"].astype(int)
veh_num2 ["age"] = veh_num2 ["age"].astype(int)
veh_num3 ["age"] = veh_num3 ["age"].astype(int)

#formatting
veh_num0 ["average_veh"] = round (veh_num0 ["average_veh"],1)
veh_num1 ["average_veh"] = round (veh_num1 ["average_veh"],1)
veh_num2 ["average_veh"] = round (veh_num2 ["average_veh"],1)
veh_num3 ["average_veh"] = round (veh_num3 ["average_veh"],1)

In [35]:
veh_num0

,incomeG,num_car,age,average_veh,income
0,1,0.00,51,0.00,8394.02
1,2,0.00,52,0.00,28544.95
2,3,0.00,43,0.00,51008.96
3,4,0.00,28,0.00,78773.02
4,5,0.00,46,0.00,261602.76


## E. Get total automobile-related expenditure for each income group at different levels of household car ownership

In [36]:
# different total car-ownership expenditures for households with 0 car
expenditure0= tx_wtexp0.groupby(['incomeG'])[["outlay_purchase", "insurance","finance",
                                             "repair","service","gas","tot_exp","population"]].sum().reset_index()
expenditure0= pd.DataFrame(expenditure0)

expenditure0["outlay_purchase"] = expenditure0 ["outlay_purchase"] / expenditure0 ["population"] 
expenditure0["finance"] = expenditure0 ["finance"] / expenditure0 ["population"] 
expenditure0["insurance"] = expenditure0 ["insurance"] / expenditure0 ["population"] 
expenditure0["repair"] = expenditure0 ["repair"] / expenditure0 ["population"] 
expenditure0["service"] = expenditure0 ["service"] / expenditure0 ["population"] 
expenditure0["gas"] = expenditure0 ["gas"] / expenditure0 ["population"] 
expenditure0["tot_exp"] = expenditure0 ["tot_exp"] / expenditure0 ["population"] 

expenditure0["Drivability (maintenance + service)"] = expenditure0 ["repair"] + expenditure0 ["service"] 
expenditure0= expenditure0.drop (columns=["repair","service"]) #drop unnecessary columns

# since households with zero cars should not have any car-ownership expense
# we need to make sure that the expense of all categories related to car spending equal to 0

expenditure0['insurance'] = 0
expenditure0['gas'] = 0
expenditure0['Drivability (maintenance + service)'] = 0


In [37]:
expenditure0

# convert the table to an image
dfi.export(
    expenditure0,
    "images/incomegroup_auto0_expenditure.png")

In [38]:
# different total car-ownership expenditures for households with one car
expenditure1 = tx_wtexp1.groupby(['incomeG'])[["outlay_purchase", "insurance","finance",
                                             "repair","service","gas","tot_exp","population"]].sum().reset_index()
expenditure1 = pd.DataFrame(expenditure1)

expenditure1 ["outlay_purchase"] = expenditure1  ["outlay_purchase"] / expenditure1  ["population"] 
expenditure1 ["finance"] = expenditure1  ["finance"] / expenditure1  ["population"] 
expenditure1 ["insurance"] = expenditure1  ["insurance"] / expenditure1  ["population"] 
expenditure1 ["repair"] = expenditure1  ["repair"] / expenditure1  ["population"] 
expenditure1 ["service"] = expenditure1  ["service"] / expenditure1  ["population"] 
expenditure1 ["gas"] = expenditure1  ["gas"] / expenditure1  ["population"] 
expenditure1 ["tot_exp"] = expenditure1  ["tot_exp"] / expenditure1  ["population"] 

expenditure1 ["Drivability (maintenance + service)"] = expenditure1  ["repair"] + expenditure1  ["service"] 
expenditure1 = expenditure1.drop (columns=["repair","service"]) #drop unnecessary columns


In [39]:
expenditure1

# convert the table to an image
dfi.export(
    expenditure1,
    "images/incomegroup_auto1_expenditure.png"
)

In [40]:
# different total car-ownership expenditures for households with two cars
expenditure2 = tx_wtexp2.groupby(['incomeG'])[["outlay_purchase", "insurance","finance",
                                             "repair","service","gas","tot_exp","population"]].sum().reset_index()
expenditure2 = pd.DataFrame(expenditure2)

expenditure2 ["outlay_purchase"] = expenditure2  ["outlay_purchase"] / expenditure2  ["population"] 
expenditure2 ["finance"] = expenditure2  ["finance"] / expenditure2  ["population"] 
expenditure2 ["insurance"] = expenditure2  ["insurance"] / expenditure2  ["population"] 
expenditure2 ["repair"] = expenditure2  ["repair"] / expenditure2  ["population"] 
expenditure2 ["service"] = expenditure2  ["service"] / expenditure2  ["population"] 
expenditure2 ["gas"] = expenditure2  ["gas"] / expenditure2  ["population"] 
expenditure2 ["tot_exp"] = expenditure2  ["tot_exp"] / expenditure2  ["population"] 

expenditure2 ["Drivability (maintenance + service)"] = expenditure2  ["repair"] + expenditure2  ["service"] 
expenditure2 = expenditure2.drop (columns=["repair","service"]) #drop unnecessary columns

In [41]:
expenditure2

# convert the table to an image
dfi.export(
    expenditure2,
    "images/incomegroup_auto2_expenditure.png")

In [42]:
# different total car-ownership expenditures for households with three or more cars
expenditure3 = tx_wtexp3.groupby(['incomeG'])[["outlay_purchase", "insurance","finance",
                                             "repair","service","gas","tot_exp","population"]].sum().reset_index()
expenditure3 = pd.DataFrame(expenditure3)

expenditure3 ["outlay_purchase"] = expenditure3  ["outlay_purchase"] / expenditure3  ["population"] 
expenditure3 ["finance"] = expenditure3  ["finance"] / expenditure3  ["population"] 
expenditure3 ["insurance"] = expenditure3  ["insurance"] / expenditure3  ["population"] 
expenditure3 ["repair"] = expenditure3  ["repair"] / expenditure3  ["population"] 
expenditure3 ["service"] = expenditure3  ["service"] / expenditure3  ["population"] 
expenditure3 ["gas"] = expenditure3  ["gas"] / expenditure3  ["population"] 
expenditure3 ["tot_exp"] = expenditure3  ["tot_exp"] / expenditure3  ["population"] 

expenditure3 ["Drivability (maintenance + service)"] = expenditure3  ["repair"] + expenditure3  ["service"] 
expenditure3 = expenditure3.drop (columns=["repair","service"]) #drop unnecessary columns

expenditure3.drop(index=expenditure3.index[0], 
        axis=0, 
        inplace=True)

In [43]:
expenditure3

# convert the table to an image
dfi.export(
    expenditure3,
    "images/incomegroup_auto3_expenditure.png")

## F. Get car-ownership expenditure (per car) for each income group at different levels of household car ownership

#### different car-ownership expenditure (per car) for households with 0 car

In [44]:
exp_perveh0  = veh_num0.merge(expenditure0, on=("incomeG"), how='left')


## calculate auto-ownership expense per year 

exp_perveh0 ['ownership_expense'] =0
exp_perveh0

,incomeG,num_car,age,average_veh,income,outlay_purchase,insurance,finance,gas,tot_exp,population,Drivability (maintenance + service),ownership_expense
0,1,0.00,51,0.00,8394.02,0.00,0,0.00,0,16000.56,426030.76,0,0
1,2,0.00,52,0.00,28544.95,0.00,0,0.00,0,24687.04,174640.03,0,0
2,3,0.00,43,0.00,51008.96,0.00,0,0.00,0,35678.07,125432.75,0,0
3,4,0.00,28,0.00,78773.02,0.00,0,0.00,0,40381.52,73976.91,0,0
4,5,0.00,46,0.00,261602.76,0.00,0,0.00,0,111234.74,56888.88,0,0


In [45]:
# Clean Data

exp_perveh0 = exp_perveh0.drop(columns=['num_car','average_veh','age', 'income','outlay_purchase','insurance','finance','population'])

In [46]:
# convert the table to an image
dfi.export(
    exp_perveh0,
    "images/incomegroup_auto0costs_percar.png")

#### different car-ownership expenditure (per car) for households with 1 car

In [47]:
exp_perveh1  = veh_num1.merge(expenditure1, on=("incomeG"), how='left')

# calculate expenditure per car
exp_perveh1[['outlay_purchase', 
            'insurance','finance','Drivability (maintenance + service)','gas']] = exp_perveh1[['outlay_purchase', 
                                                         'insurance','finance','Drivability (maintenance + service)','gas']].div(exp_perveh1.average_veh, axis=0)


## calculate auto-ownership expense per year 

exp_perveh1['ownership_expense'] =exp_perveh1['average_veh'] * (exp_perveh1['outlay_purchase'] + exp_perveh1['finance']
                                                            + exp_perveh1['insurance'])


exp_perveh1 = exp_perveh1.round(2) # formatting
exp_perveh1 

,incomeG,num_car,age,average_veh,income,outlay_purchase,insurance,finance,gas,tot_exp,population,Drivability (maintenance + service),ownership_expense
0,1,1.00,54,1.00,9644.11,1326.57,945.72,101.87,1064.48,27842.57,946121.83,679.72,2374.16
1,2,1.00,50,1.00,29207.42,1168.95,1144.45,140.78,1583.04,33429.40,1104226.74,851.41,2454.18
2,3,1.00,44,1.00,48520.05,1419.10,1352.58,181.72,1859.75,41005.24,638315.77,648.28,2953.41
3,4,1.00,42,1.00,75228.16,2866.98,2066.57,251.45,1971.93,56656.74,644260.31,1490.71,5185.00
4,5,1.00,45,1.00,147255.47,3289.25,2159.52,361.22,2459.62,74599.21,443826.22,1652.27,5809.99


In [48]:
# Clean Data

exp_perveh1 = exp_perveh1.drop(columns=['num_car','average_veh','age', 'income','outlay_purchase','insurance','finance','population'])

In [49]:
exp_perveh1

# convert the table to an image
dfi.export(
    exp_perveh1,
    "images/incomegroup_auto1costs_percar.png")

#### different car-ownership expenditure (per car) for households with two cars


In [50]:
exp_perveh2  = veh_num2.merge(expenditure2, on=("incomeG"), how='left')

# calculate expenditure per car
exp_perveh2[['outlay_purchase', 
            'insurance','finance','Drivability (maintenance + service)','gas']] = exp_perveh2[['outlay_purchase', 
                                                         'insurance','finance','Drivability (maintenance + service)','gas']].div(exp_perveh2.average_veh, axis=0)


## calculate auto-ownership expense per year 

exp_perveh2['ownership_expense'] =exp_perveh2['average_veh'] * (exp_perveh2['outlay_purchase'] + exp_perveh2['finance']
                                                            + exp_perveh2['insurance'])
exp_perveh2 = exp_perveh2.round(2) # formatting
exp_perveh2 

,incomeG,num_car,age,average_veh,income,outlay_purchase,insurance,finance,gas,tot_exp,population,Drivability (maintenance + service),ownership_expense
0,1,2.00,63,2.00,10323.95,1473.51,746.25,118.95,846.12,33492.70,255058.19,232.20,4677.42
1,2,2.00,57,2.00,28812.51,1132.47,830.05,125.02,1047.14,41066.02,660803.63,651.62,4175.07
2,3,2.00,50,2.00,48813.01,1484.48,1015.14,120.65,1181.75,49929.68,647069.10,453.83,5240.54
3,4,2.00,45,2.00,75991.22,4591.32,1081.48,180.17,1325.61,60094.37,905342.91,567.54,11705.95
4,5,2.00,46,2.00,217724.15,2728.39,1398.98,301.72,1533.08,106119.81,1222959.93,872.64,8858.19


In [51]:
# Clean Data

exp_perveh2 = exp_perveh2.drop(columns=['num_car','average_veh','age', 'income','outlay_purchase','insurance','finance','population'])

In [52]:
exp_perveh2

# convert the table to an image
dfi.export(
    exp_perveh2,
    "images/incomegroup_auto2costs_percar.png")

#### different car-ownership expenditure (per car) for households with three or more cars


In [53]:
exp_perveh3  = veh_num3.merge(expenditure3, on=("incomeG"), how='left')

# calculate expenditure per car
exp_perveh3[['outlay_purchase', 
            'insurance','finance','Drivability (maintenance + service)','gas']] = exp_perveh3[['outlay_purchase', 
                                                         'insurance','finance','Drivability (maintenance + service)','gas']].div(exp_perveh3.average_veh, axis=0)


## calculate auto-ownership expense per year 

exp_perveh3['ownership_expense'] =exp_perveh3['average_veh'] * (exp_perveh3['outlay_purchase'] + exp_perveh3['finance']
                                                            + exp_perveh3['insurance'])
exp_perveh3 = exp_perveh3.round(2) # formatting
exp_perveh3 

,incomeG,num_car,age,average_veh,income,outlay_purchase,insurance,finance,gas,tot_exp,population,Drivability (maintenance + service),ownership_expense
0,1,3.00,51,3.00,11003.02,1860.64,893.74,192.40,746.94,46013.49,62555.77,356.21,8840.33
1,2,3.51,61,3.50,30237.16,2085.97,602.37,79.77,812.93,49059.07,279060.97,163.99,9688.41
2,3,3.72,54,3.70,49262.44,2934.09,708.70,72.53,589.02,54427.33,292997.23,222.57,13746.70
3,4,3.94,54,3.90,79952.11,1905.22,704.59,158.27,959.84,83945.65,401059.13,301.65,10795.51
4,5,3.68,49,3.70,197068.98,2727.82,866.02,230.50,966.79,115088.60,1263496.95,709.63,14150.09


In [54]:
# Clean Data

exp_perveh3 = exp_perveh3.drop(columns=['num_car','average_veh','age', 'income','outlay_purchase','insurance','finance','population'])

In [55]:
exp_perveh3

# convert the table to an image
dfi.export(
    exp_perveh3,
    "images/incomegroup_auto3costs_percar.png")

## G. Use CES estimated expenditure table (above) as a reference, to calculate auto-related expenses (i.e., car-ownership costs and driving costs) at ACS's income segregation level and corresponding number of vehicles owned

### G.1) obtain auto-ownership owned per households at ACS's income segragation level, at the tract Level

The level of car-ownership information (e.g., 1, or 2, or 3 vehicles available) is only available at tract level, we thus first need to assign CES estimated expenditures to the corresponding income & car-ownership group at the ACS tract level:

ACS Variables needed:

* median household income  = "B19013_001E",
* the number of households = "B11001_001E", 
* aggregate number of vehicles available = "B25046_001E",
* no vehicle available = "B08141_002E"
* 1 vehicle available = "B08141_003E"
* 2 vehicle available = "B08141_004E"
* 3 vehicle available = "B08141_005E"

In [56]:

###Load Campo region's ACS Data at the tract level###
tract_census = c.acs5.state_county_tract(fields = ('NAME', 'B19013_001E','B11001_001E','B08141_002E','B08141_003E','B08141_004E','B08141_005E'), 
                                            state_fips = states.TX.fips,
                                            county_fips = "*",
                                            tract = "*",
                                            year = 2019)

# Create a dataframe for the downloaded census data
tract_census = pd.DataFrame(tract_census)

# rename each variables with more intuitive names
column_names = ['NAME', 'medincome_hh','hh_tract','veh0','veh1','veh2','veh3',
                'state','county','tract']

tract_census.columns = column_names

county = ['021', '053', '055','209','453','491']
tract_census = tract_census[tract_census['county'].isin(county)] 


tract_census [tract_census['medincome_hh']<0] = 0


#### create GEOIDs for each tract
# Combine state, county and tract columns together to create a new string and assign to new column

tract_census["geoid"] = tract_census["state"] + tract_census["county"] + tract_census ["tract"]

# Remove columns that are no longer needed
tract_census = tract_census.drop(columns = ["state", "county","tract","NAME"])

# change the order of the column
cols = tract_census.columns.tolist()
cols = cols[-1:] + cols[:-1] #move "geoid" column forward to the front
tract_census = tract_census[cols] #apply the new column sequence 

In [57]:
#### create the income segregation for ACS data at the tract level

tract_census["incomeG"] = np.select(
    [
        tract_census ['medincome_hh'].between(0, 20000, inclusive='left'), 
        tract_census ['medincome_hh'].between(20000, 39999, inclusive='both'),
        tract_census ['medincome_hh'].between(40000, 59999, inclusive='both'), 
        tract_census ['medincome_hh'].between(60000, 99999, inclusive='both'),
        tract_census ['medincome_hh'].between(100000, 100000000, inclusive='left')
    ], 
    [
        '1', 
        '2',
        '3', 
        '4',
        '5'
    ]
)

# tract_census.query("incomeG == '1'").shape[0] # double check the number of rows in each income segragation


In [58]:
tract_census

,geoid,medincome_hh,hh_tract,veh0,veh1,veh2,veh3,incomeG
39,48453002110,46265.00,1366.00,91.00,505.00,694.00,975.00,3
40,48453002315,37843.00,1479.00,143.00,1048.00,775.00,165.00,2
103,48453001835,60048.00,2435.00,0.00,951.00,1638.00,1106.00,4
104,48453001840,68520.00,3961.00,95.00,1010.00,2957.00,1992.00,4
105,48453001850,60253.00,1933.00,138.00,1116.00,863.00,390.00,4
...,...,...,...,...,...,...,...,...
5169,48453001771,133077.00,1838.00,20.00,207.00,1073.00,733.00,5
5170,48453001772,81307.00,1609.00,0.00,449.00,1279.00,996.00,4
5171,48453001773,135493.00,3391.00,20.00,430.00,2566.00,983.00,5
5183,48053960700,47411.00,3171.00,65.00,1246.00,1018.00,858.00,3


In [59]:
### cross-table : assign CES estimated expenditures to the corresponding income & car-ownership group at the ACS tract level###


### join ACS data with CES expenditure estimates dataframe

# Assign carownership expenditures and total household expenditure estimated by CES to those income groups with different level of car ownership 

tract_carcost = tract_census.merge(exp_perveh0,how='left',on = 'incomeG').merge(exp_perveh1,how='left',on = 'incomeG')

tract_carcost ['tot_exp_veh0'] =  tract_carcost ['veh1'] * tract_carcost ['tot_exp_x'] 

tract_carcost ['tot_exp_veh1'] =  tract_carcost ['veh1'] * tract_carcost ['tot_exp_y'] 
tract_carcost ['ownership_expense_veh1'] =  tract_carcost ['veh1'] * tract_carcost ['ownership_expense_y'] 
tract_carcost ['gas_veh1'] =  tract_carcost ['veh1'] * tract_carcost ['gas_y'] 
tract_carcost ['drivability_veh1'] =  tract_carcost ['veh1'] * tract_carcost ['Drivability (maintenance + service)_y'] 

tract_carcost = tract_carcost.drop(columns=['tot_exp_x','ownership_expense_x','Drivability (maintenance + service)_x','gas_x',
                                           'tot_exp_y','ownership_expense_y','Drivability (maintenance + service)_y','gas_y'])# clean data



tract_carcost = tract_carcost.merge(exp_perveh2,how='left',on = 'incomeG').merge(exp_perveh3,how='left',on = 'incomeG')

tract_carcost ['tot_exp_veh2'] =  tract_carcost ['veh2'] * tract_carcost ['tot_exp_x'] 
tract_carcost ['ownership_expense_veh2'] =  tract_carcost ['veh2'] * tract_carcost ['ownership_expense_x'] 
tract_carcost ['gas_veh2'] =  tract_carcost ['veh2'] * tract_carcost ['gas_x'] 
tract_carcost ['drivability_veh2'] =  tract_carcost ['veh2'] * tract_carcost ['Drivability (maintenance + service)_x'] 


tract_carcost ['tot_exp_veh3'] =  tract_carcost ['veh3'] * tract_carcost ['tot_exp_y'] 
tract_carcost ['ownership_expense_veh3'] =  tract_carcost ['veh3'] * tract_carcost ['ownership_expense_y'] 
tract_carcost ['gas_veh3'] =  tract_carcost ['veh3'] * tract_carcost ['gas_y'] 
tract_carcost ['drivability_veh3'] =  tract_carcost ['veh3'] * tract_carcost ['Drivability (maintenance + service)_y'] 

tract_carcost = tract_carcost.drop(columns=['tot_exp_x','ownership_expense_x','Drivability (maintenance + service)_x','gas_x',
                                           'tot_exp_y','ownership_expense_y','Drivability (maintenance + service)_y','gas_y'])# clean data

tract_carcost

,geoid,medincome_hh,hh_tract,veh0,veh1,veh2,veh3,incomeG,tot_exp_veh0,tot_exp_veh1,...,gas_veh1,drivability_veh1,tot_exp_veh2,ownership_expense_veh2,gas_veh2,drivability_veh2,tot_exp_veh3,ownership_expense_veh3,gas_veh3,drivability_veh3
0,48453002110,46265.00,1366.00,91.00,505.00,694.00,975.00,3,18017422.88,20707646.20,...,939173.75,327381.40,34651197.92,3636934.76,820134.50,314958.02,53066646.75,13403032.50,574294.50,217005.75
1,48453002315,37843.00,1479.00,143.00,1048.00,775.00,165.00,2,25872016.47,35034011.20,...,1659025.92,892277.68,31826165.50,3235679.25,811533.50,505005.50,8094746.55,1598587.65,134133.45,27058.35
2,48453001835,60048.00,2435.00,0.00,951.00,1638.00,1106.00,4,38402826.92,53880559.74,...,1875305.43,1417665.21,98434578.06,19174346.10,2171349.18,929630.52,92843888.90,11939834.06,1061583.04,333624.90
3,48453001840,68520.00,3961.00,95.00,1010.00,2957.00,1992.00,4,40785336.69,57223307.40,...,1991649.30,1505617.10,177699052.09,34614494.15,3919828.77,1678215.78,167219734.80,21504655.92,1912001.28,600886.80
4,48453001850,60253.00,1933.00,138.00,1116.00,863.00,390.00,4,45065777.96,63228921.84,...,2200673.88,1663632.36,51861441.31,10102234.85,1144001.43,489787.02,32738803.50,4210248.90,374337.60,117643.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,48453001771,133077.00,1838.00,20.00,207.00,1073.00,733.00,5,23025590.16,15442036.47,...,509141.34,342019.89,113866556.13,9504837.87,1644994.84,936342.72,84359943.80,10372015.97,708657.07,520158.79
354,48453001772,81307.00,1609.00,0.00,449.00,1279.00,996.00,4,18131303.14,25438876.26,...,885396.57,669328.79,76860699.23,14971910.05,1695455.19,725883.66,83609867.40,10752327.96,956000.64,300443.40
355,48453001773,135493.00,3391.00,20.00,430.00,2566.00,983.00,5,47830936.08,32077660.30,...,1057636.60,710476.10,272303432.46,22730115.54,3933883.28,2239194.24,113132093.80,13909538.47,950354.57,697566.29
356,48053960700,47411.00,3171.00,65.00,1246.00,1018.00,858.00,3,44454869.13,51092529.04,...,2317248.50,807756.88,50828414.24,5334869.72,1203021.50,461998.94,46698649.14,11794668.60,505379.16,190965.06


In [60]:
tract_carcost.describe()

,medincome_hh,hh_tract,veh0,veh1,veh2,veh3,tot_exp_veh0,tot_exp_veh1,ownership_expense_veh1,gas_veh1,drivability_veh1,tot_exp_veh2,ownership_expense_veh2,gas_veh2,drivability_veh2,tot_exp_veh3,ownership_expense_veh3,gas_veh3,drivability_veh3
count,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00,358.00
mean,80994.97,2183.61,65.85,663.39,1434.97,987.07,32194742.31,35536931.96,2988713.03,1315572.18,830996.94,101206159.75,13681900.96,1934703.48,912410.39,83947033.13,12042970.91,872591.73,382654.98
std,35372.34,1101.76,89.46,495.76,891.40,764.50,29896181.46,27585245.30,2414652.29,979889.65,700918.93,85159535.77,10107485.52,1305635.28,705195.34,72714697.18,9457776.66,722080.15,392726.23
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,57682.50,1447.75,4.00,277.75,823.00,456.75,14396012.40,15487276.46,1330087.74,575983.30,349637.81,49463328.10,6201022.59,1058464.67,467085.42,33510281.15,5206134.70,370494.74,131066.92
50%,74407.50,2030.00,33.00,559.50,1242.50,732.00,25003211.27,30077343.54,2390518.67,1106345.90,647248.21,76501283.73,11077712.20,1658470.56,721636.58,59296108.51,9376564.28,634454.24,223829.70
75%,95884.25,2690.75,84.25,911.50,1799.50,1315.00,42721271.85,48469245.90,3927914.78,1807766.83,1089559.36,127821311.14,18504180.46,2468734.40,1166862.24,116768399.15,16555605.30,1181938.93,511394.92
max,223424.00,7662.00,488.00,3243.00,7722.00,4701.00,360734245.83,241925238.03,18841797.57,7976547.66,5358311.61,819457172.82,68402943.18,11838443.76,6738526.08,455405590.20,55991906.13,4512207.84,2808005.91


In [61]:
# calculate the weighted car-related expenditure and total household expenditure, based on the weights of veh-ownership population 

tract_carcost ['veh_tot'] = tract_carcost ['veh0']+tract_carcost ['veh1']+tract_carcost ['veh2']+tract_carcost ['veh3']
tract_carcost ['weight0']=tract_carcost ['veh0']/tract_carcost ['veh_tot']
tract_carcost ['weight1']=tract_carcost ['veh1']/tract_carcost ['veh_tot']
tract_carcost ['weight2']=tract_carcost ['veh2']/tract_carcost ['veh_tot']
tract_carcost ['weight3']=tract_carcost ['veh3']/tract_carcost ['veh_tot']


tract_carcost ['tot_exp'] = ((tract_carcost ['tot_exp_veh0'] * tract_carcost ['weight0']) + (tract_carcost ['tot_exp_veh1'] * tract_carcost ['weight1'] )+ (tract_carcost ['tot_exp_veh2'] * tract_carcost ['weight2']) +(tract_carcost ['tot_exp_veh3'] * tract_carcost ['weight3']))/tract_carcost ['hh_tract']
tract_carcost ['carownership_exp_hh'] = ((tract_carcost ['ownership_expense_veh1'] * tract_carcost ['weight1']) + (tract_carcost ['ownership_expense_veh2'] * tract_carcost ['weight2']) +(tract_carcost ['ownership_expense_veh3'] * tract_carcost ['weight3']))/tract_carcost ['hh_tract']
tract_carcost ['gas_hh'] = ((tract_carcost ['gas_veh1'] * tract_carcost ['weight1']) + (tract_carcost ['gas_veh2'] * tract_carcost ['weight2']) + (tract_carcost ['gas_veh3'] * tract_carcost ['weight3']))/tract_carcost ['hh_tract']
tract_carcost ['drivability_hh'] = ((tract_carcost ['drivability_veh1'] * tract_carcost ['weight1']) + (tract_carcost ['drivability_veh2'] * tract_carcost ['weight2']) +(tract_carcost ['drivability_veh3'] * tract_carcost ['weight3']))/ tract_carcost ['hh_tract']


# clean data

tract_carcost = tract_carcost[['geoid', 'hh_tract','tot_exp','carownership_exp_hh','gas_hh','drivability_hh']]


In [62]:
tract_carcost.describe()

,hh_tract,tot_exp,carownership_exp_hh,gas_hh,drivability_hh
count,358.00,355.00,355.00,355.00,355.00
mean,2183.61,40584.55,5330.29,722.12,357.03
std,1101.76,17989.28,1986.45,167.99,136.02
min,0.00,7770.33,752.98,154.28,60.42
25%,1447.75,27675.11,4076.09,611.56,246.67
50%,2030.00,35499.28,5388.92,731.38,340.31
75%,2690.75,50499.83,6784.36,822.81,442.31
max,7662.00,94633.17,10629.51,1207.65,763.04


### G.2) Assign weighted car-related expense from tract level to block group Level

In [63]:
# Set API key
c = Census("bc23309be09398b025511fe928c4840534171582") #you need to apply yours online

#Load Campo ACS Data at the block groups level
bgs_census = c.acs5.state_county_blockgroup(fields = ('NAME', 'B11001_001E','B19013_001E'), # B11001_001E # all types of households
                                            state_fips = states.TX.fips,
                                            county_fips = "*",
                                            blockgroup = "*",
                                            year = 2019)
# Create a dataframe for the downloaded census data
bgs_census = pd.DataFrame(bgs_census)

# rename each variables with more intuitive names
column_names = ['NAME','hh_bgs','medincome','state','county','tract','block group']

bgs_census.columns = column_names

# filter needed county 
# (CAMPO's counties fips:#Bastrop =021,Burnet=053,Caldwell=055,Hays =209,Travis=453,Williamson = 491)

county = ['021', '053', '055','209','453','491']
bgs_census = bgs_census[bgs_census['county'].isin(county)] 

#### create GEOIDs for each block group
# Combine state, county, tract and block group columns together to create a new string and assign to new column

bgs_census["geoid_bgs"] = bgs_census["state"] + bgs_census["county"] + bgs_census ["tract"] + bgs_census ["block group"]
bgs_census["geoid_tract"] = bgs_census["state"] + bgs_census["county"] + bgs_census ["tract"]

# Remove columns that are no longer needed
bgs_census = bgs_census.drop(columns = ["state", "county","tract","block group","NAME"])

# change the order of the column
cols = bgs_census.columns.tolist()
cols = cols[-1:] + cols[:-1] #move "geoid" column forward to the front
bgs_census = bgs_census[cols] #apply the new column sequence 

bgs_census

,geoid_tract,hh_bgs,medincome,geoid_bgs
196,48021950100,539.00,88025.00,480219501001
197,48021950100,680.00,88472.00,480219501002
198,48021950100,637.00,58909.00,480219501003
199,48021950100,477.00,79485.00,480219501004
200,48021950100,354.00,63542.00,480219501005
...,...,...,...,...
15664,48491021601,1145.00,72292.00,484910216013
15665,48491021602,462.00,72153.00,484910216021
15666,48491021602,316.00,70476.00,484910216022
15667,48491021603,197.00,83438.00,484910216031


In [64]:
# join data frame of the auto-related expenditure at the tract level and THE ACS block group data frame 

tract_carcost.geoid = tract_carcost.geoid.astype(str)
bgs_census.geoid_trac= bgs_census.geoid_tract.astype(str)

carcost = bgs_census.merge(tract_carcost, how = 'left',left_on = 'geoid_tract',right_on = 'geoid')

carcost ["ratio (drivability to fuel cost)"] = carcost ["drivability_hh"] / carcost["gas_hh"]

carcost = carcost.drop (columns = ['geoid','geoid_tract','drivability_hh','gas_hh'])

carcost.rename(columns={'geoid_bgs': 'geoid'}, inplace=True)


In [65]:
#### assign income segregation at the block groups level 
carcost["incomeG"] = np.select(
    [
        carcost ['medincome'].between(0, 20000, inclusive='left'), 
        carcost ['medincome'].between(20000, 39999, inclusive='both'),
        carcost ['medincome'].between(40000, 59999, inclusive='both'), 
        carcost ['medincome'].between(60000, 99999, inclusive='both'),
        carcost ['medincome'].between(100000, 100000000, inclusive='left')
    ], 
    [
        '1', 
        '2',
        '3', 
        '4',
        '5'
    ]
)

# carcost.query("incomeG == '1'").shape[0] # double check the number of rows in each income segragation

In [66]:
carcost 

,hh_bgs,medincome,geoid,hh_tract,tot_exp,carownership_exp_hh,ratio (drivability to fuel cost),incomeG
0,539.00,88025.00,480219501001,2687.00,39960.49,5742.03,0.39,4
1,680.00,88472.00,480219501002,2687.00,39960.49,5742.03,0.39,4
2,637.00,58909.00,480219501003,2687.00,39960.49,5742.03,0.39,3
3,477.00,79485.00,480219501004,2687.00,39960.49,5742.03,0.39,4
4,354.00,63542.00,480219501005,2687.00,39960.49,5742.03,0.39,4
...,...,...,...,...,...,...,...,...
989,1145.00,72292.00,484910216013,1580.00,50624.86,7849.18,0.39,4
990,462.00,72153.00,484910216021,778.00,54991.17,7725.45,0.36,4
991,316.00,70476.00,484910216022,778.00,54991.17,7725.45,0.36,4
992,197.00,83438.00,484910216031,495.00,49010.87,6841.47,0.39,4


### G.3) Calculate driving costs based on VMT produced at block group level

In [67]:
###assign VMT to each bgs, by using 
vmt = vmt [['geoid','tot_vmthh_bgs_year']]

carcost.geoid = carcost.geoid.astype(str)
vmt.geoid = vmt.geoid.astype(str)

campo_autoexp = vmt.merge(carcost, how = 'left',on = 'geoid')

campo_autoexp.head(2)

,geoid,tot_vmthh_bgs_year,hh_bgs,medincome,hh_tract,tot_exp,carownership_exp_hh,ratio (drivability to fuel cost),incomeG
0,480539607001,11259.95,606.00,78521.00,3171.00,15670.10,1992.48,0.36,4
1,480539603002,18786.28,393.00,55838.00,2122.00,29789.21,4733.53,0.38,3


In [68]:
mpg = 22.07  # assign the average fuel efficiency in Texas for 2019 (Resource: NHTS)
G = 2.691  # assign the cost of gas per gallon (average annual regional cost in Gulf Coast for 2019) 

In [69]:
# driving cost cost per year

campo_autoexp ['driving_cost_year'] = (campo_autoexp ['tot_vmthh_bgs_year']/mpg) * G * (1+campo_autoexp ['ratio (drivability to fuel cost)'])    
campo_autoexp ['driving_cost_year'] = campo_autoexp ['driving_cost_year'].fillna(0)


# A brief summary for the driving costs asscoaited with different income groups
a = campo_autoexp.groupby('incomeG', as_index=False).agg(vmt=('tot_vmthh_bgs_year','mean'), 
                                            ratio= ('ratio (drivability to fuel cost)','mean'),
                                            driving_cost= ('driving_cost_year','mean'))
a.drop(index=a.index[0], 
        axis=0, 
        inplace=True)

# convert the table to an image
dfi.export(
    a,
    "images/incomegroup_drivingcosts.png")

In [70]:
campo_autoexp

,geoid,tot_vmthh_bgs_year,hh_bgs,medincome,hh_tract,tot_exp,carownership_exp_hh,ratio (drivability to fuel cost),incomeG,driving_cost_year
0,480539607001,11259.95,606.00,78521.00,3171.00,15670.10,1992.48,0.36,4,1868.46
1,480539603002,18786.28,393.00,55838.00,2122.00,29789.21,4733.53,0.38,3,3158.99
2,480539601002,33210.16,698.00,80455.00,2285.00,47803.77,6944.10,0.38,4,5572.87
3,480539608001,32868.98,695.00,116103.00,2364.00,36257.96,5624.76,0.41,5,5647.91
4,480539604003,28047.42,940.00,61893.00,2135.00,20608.05,3740.34,0.38,4,4721.52
...,...,...,...,...,...,...,...,...,...,...
989,484530017512,12320.61,690.00,137574.00,972.00,49972.12,4477.10,0.59,5,2389.10
990,484530017522,6508.35,734.00,61506.00,2171.00,21212.12,1716.34,0.36,4,1081.01
991,484530017462,6718.91,614.00,49167.00,2026.00,31785.83,5169.04,0.53,3,1256.84
992,484530017473,7069.80,602.00,76058.00,2204.00,39266.07,6495.68,0.42,4,1226.31


In [71]:
# maintain necessary varaibles
campo_autoexp1 = campo_autoexp [['geoid','tot_vmthh_bgs_year','hh_bgs','carownership_exp_hh','driving_cost_year','tot_exp']]

campo_autoexp1.rename(columns={'tot_vmthh_bgs_year': 'all_vmt_hh','hh_bgs':'hh','carownership_exp_hh':'carownership_expense_hh','tot_exp':'annual_expense_individual' }, inplace=True)

campo_autoexp1.to_csv('autoexp.csv',index=False) # export to the csv file